# Point Spectrum

Given a matrix $B$, let $\chi_B$ denote the characteristic polynomial.

For a point $E$ to lay in the point spectrum of $H$ (cf. Proposition 3.8) we need that
$$
\chi_{H_{0..K-2}}(E) = 0
$$
and
$$
-1 < \chi_{H_{0..K-1}}(E) < 1.
$$

According to Remark 3.9 these values correspond to the entries $M_{2,1}(E)$ and $M_{1,1}(E)$ of the monodromy matrix.

In [ ]:
# Run this cell once at startup

l = var('l')
E = var('E')

def mon(v):
#
# Calculate the general monodromy matrix with scaling l #
# and energy E
#
# Example:
# v = [1,1,0]; mon(v).expand()
# > [E^2 - E*l - 1            -E]
#   [        E - l            -1]
#
##########################
    M = identity_matrix(2)
    for vv in v:
        M = matrix(2,2,[E - l*vv, -1, 1, 0])*M
    return M

def lim_pots(v):
#
# determine all unique limit potentials of a given
# periodic potential v by considering all shifts and flips
#
##########################
    v_rev = deepcopy(v)
    lim = shift_pots(v)
    v_rev.reverse()
    lim_rev = shift_pots(v_rev)
    for pot in lim_rev:
        if pot not in lim:
            lim.append(pot)
    return lim
    
def shift_pots(v):
#
# determine all unique shifts of a given periodic potential v
#
##########################
    shift = []
    for c in range(len(v)):
        if v not in shift:
            shift.append(v)
        v = right_shift(v)
    return shift
    
def right_shift(a):
    return [a[-1] , *a[:-1]]


In [ ]:
# this cell contains basic routines that need to be loaded 
# before data generation can start

def analyse_v(v, ll):
##########################
# calculate monodromy, zeros and check trace. 
# return a dictionary containing symbolic elements
# 
# This routine analyses the scaling l = ll.
#
# example:
# v = [1,1,0]; analyse_v(v,0.1)
##########################
    data = {}
#monodromy
    M = mon(v).expand() 
#zeros
    sols = solve(M[1][0].subs(l=ll) == 0, E, to_poly_solve=True)
    sol = list(map(lambda s: real(s.rhs().n()), sols))
    sol.sort()
    data['sol'] = sol
    data['cond'] = list(map(lambda s: real(M[0,0].subs(l=ll).subs(E=s).n()), sol))
    return data
   
def build_dict(v,nsteps,start,end):
    v_dict = {}
    for s in range(nsteps + 1):
        ll = start + end * s/nsteps
        v_dict[ll] =  analyse_v(v, ll)
    return v_dict
        
def hagger_filter(H, eps=0.01):
    isev = {}
    traj = {}
    for ev in range(len(H[0]['sol'])):
        isev[ev] = False
        for ll in H:
            if(abs(H[ll]['cond'][ev].n()) < 1 - eps):
                isev[ev] = True
                print("Eigenvalue trajectory found: " + "_" + str(ev))
                break
        traj[ev] = []
        for ll in H:
            traj[ev].append([ll.n(),H[ll]['sol'][ev].n(),(H[ll]['cond'][ev].n()) ])
    return traj

import csv
def export_traj(v,H):
    for ev in H:
        prefix = "data/"
        outname = prefix + str(list(v)).replace(", ", "").replace("[", "").replace("]", "") + "_" +  str(ev) + ".csv"
        f = open(outname, 'w')
        writefile = csv.writer(f)
        for line in H[ev]:
            writefile.writerow(line)
        f.close()
        

# K4

For the potential $v = (1,1,0,1)$, we determine the point spectra of all limit potentials of $v$. The algorithm first determines all candidates for eigenvalues, calculating a trajectory $\gamma(l)$ of the corresponding zeros of the monodromy matrix entry.

Trajectories are exported into `.csv` files. The visual output indicates whether a trajectory consists of eigenvalues or not. See the Octave postprocessing routines for the files that are used for the plots

In [ ]:
v = [1,1,0,1]
v_list = lim_pots(v)
H = {}
nsteps =150
start = 0
end = 1.5
trajectories = {}

for vv in v_list:
    print("Checking potential:", vv)
    H[tuple(vv)] = build_dict(vv,nsteps,start,end)
    trajectories[tuple(vv)] = hagger_filter(H[tuple(vv)])
    export_traj(vv,trajectories[tuple(vv)])


# K5

In [ ]:
v = [1,1,0,1,0]
v_list = lim_pots(v)
H = {}
nsteps =150
start = 0
end = 1.5
trajectories = {}

for vv in v_list:
    print("Checking potential:", vv)
    H[tuple(vv)] = build_dict(vv,nsteps,start,end)
    trajectories[tuple(vv)] = hagger_filter(H[tuple(vv)])
    export_traj(vv,trajectories[tuple(vv)])


# K9

In [ ]:
v = [1,1,0,1,0,1,0,1,1]
v_list = lim_pots(v)
H = {}
nsteps =100
start = 0
end = 2.5
trajectories = {}

for vv in v_list:
    print("Checking potential:", vv)
    H[tuple(vv)] = build_dict(vv,nsteps,start,end)
    trajectories[tuple(vv)] = hagger_filter(H[tuple(vv)])
    export_traj(vv,trajectories[tuple(vv)])
